In [1]:
from keras.datasets import imdb

# Load the data, keeping only 10,000 of the most frequently occuring words
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

17464789/17464789 [==============================] - 1s 0us/step


In [2]:
# Here is a list of maximum indexes in every review --- we search the maximum index in this list of max indexes
print(type([max(sequence) for sequence in train_data]))

# Find the maximum of all max indexes
max([max(sequence) for sequence in train_data])

<class 'list'>


9999

In [3]:
# step 1: load the dictionary mappings from word to integer index
word_index = imdb.get_word_index()

# step 2: reverse word index to map integer indexes to their respective words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Step 3: decode the review, mapping integer indices to words
#
# indices are off by 3 because 0, 1, and 2 are reserverd indices for "padding", "Start of sequence" and "unknown"
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])

decoded_review

1641221/1641221 [==============================] - 1s 0us/step


"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [4]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))    # Creates an all zero matrix of shape (len(sequences),10K)
    for i,sequence in enumerate(sequences):
        results[i,sequence] = 1                        # Sets specific indices of results[i] to 1s
    return results

# Vectorize training Data
X_train = vectorize_sequences(train_data)

# Vectorize testing Data
X_test = vectorize_sequences(test_data)

In [5]:
X_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

In [6]:
X_train.shape

(25000, 10000)

In [7]:
y_train = np.asarray(train_labels).astype('float32')
y_test  = np.asarray(test_labels).astype('float32')

In [8]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [9]:
from keras import optimizers
from keras import losses
from keras import metrics

model.compile(optimizer=optimizers.RMSprop(lr=0.001),loss = losses.binary_crossentropy,metrics = [metrics.binary_accuracy])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [10]:
# Input for Validation
X_val = X_train[:10000]
partial_X_train = X_train[10000:]

# Labels for validation
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [11]:
history = model.fit(partial_X_train, partial_y_train, epochs=20, batch_size=512, validation_data=(X_val, y_val))

Epoch 1/20
30/30 [==============================] - 4s 96ms/step - loss: 0.5801 - binary_accuracy: 0.7857 - val_loss: 0.4867 - val_binary_accuracy: 0.8178
Epoch 2/20
30/30 [==============================] - 1s 40ms/step - loss: 0.3871 - binary_accuracy: 0.8913 - val_loss: 0.3540 - val_binary_accuracy: 0.8822
Epoch 3/20
30/30 [==============================] - 1s 41ms/step - loss: 0.2782 - binary_accuracy: 0.9133 - val_loss: 0.2939 - val_binary_accuracy: 0.8913
Epoch 4/20
30/30 [==============================] - 1s 40ms/step - loss: 0.2084 - binary_accuracy: 0.9300 - val_loss: 0.2760 - val_binary_accuracy: 0.8914
Epoch 5/20
30/30 [==============================] - 1s 39ms/step - loss: 0.1678 - binary_accuracy: 0.9461 - val_loss: 0.2760 - val_binary_accuracy: 0.8905
Epoch 6/20
30/30 [==============================] - 1s 40ms/step - loss: 0.1380 - binary_accuracy: 0.9585 - val_loss: 0.2857 - val_binary_accuracy: 0.8855
Epoch 7/20
30/30 [==============================] - 1s 38ms/step - los

In [12]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'binary_accuracy', 'val_loss', 'val_binary_accuracy'])

In [13]:
model.fit(partial_X_train, partial_y_train, epochs=3, batch_size=512, validation_data=(X_val, y_val))

Epoch 1/3
30/30 [==============================] - 2s 73ms/step - loss: 0.0062 - binary_accuracy: 0.9997 - val_loss: 0.6923 - val_binary_accuracy: 0.8660
Epoch 2/3
30/30 [==============================] - 1s 41ms/step - loss: 0.0036 - binary_accuracy: 0.9998 - val_loss: 0.7613 - val_binary_accuracy: 0.8634
Epoch 3/3
30/30 [==============================] - 1s 39ms/step - loss: 0.0042 - binary_accuracy: 0.9996 - val_loss: 0.7715 - val_binary_accuracy: 0.8608


In [14]:
# Making Predictions for testing data
np.set_printoptions(suppress=True)
result = model.predict(X_test)
result

782/782 [==============================] - 2s 2ms/step


array([[0.00575977],
       [1.        ],
       [0.24155231],
       ...,
       [0.0024741 ],
       [0.00291688],
       [0.6468026 ]], dtype=float32)

In [15]:
from sklearn.metrics import mean_absolute_error
y_pred = np.zeros(len(result))
for i, score in enumerate(result):
    y_pred[i] = 1 if score > 0.5 else 0

mae = mean_absolute_error(y_pred, y_test)
mae

0.15428